In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium

pd.set_option('display.max_rows', 10)

In [4]:
## 인구

pop = pd.read_csv('./data/23년 주민등록인구.csv', skiprows=2)
pop= pop[['동별(2)', '동별(3)', '소계.1']]
pop.columns = ['행정구', '행정동', '인구수']
pop.drop( pop[pop['행정동'] == '소계'].index, axis=0, inplace = True)
pop.sort_values('행정동', inplace = True)
pop.reset_index(drop= True, inplace = True);pop

,행정구,행정동,인구수
0,송파구,가락1동,27670
1,송파구,가락2동,31450
2,송파구,가락본동,25617
3,구로구,가리봉동,12478
4,금천구,가산동,27166
...,...,...,...
421,용산구,효창동,10828
422,용산구,후암동,16540
423,동대문구,휘경1동,16168
424,동대문구,휘경2동,25288


In [5]:
## 산업지역

work = pd.read_csv('./data/21년 서울시 사업체현황.csv')
work_company = work[(work['동별'].str[-1] == '동') & (work['구분별'] == '사업체수') & (work['산업대분류별'] == '합계')]
worker = work[(work['동별'].str[-1] == '동') & (work['구분별'] == '종사자수') & (work['산업대분류별'] == '합계')]

worker = worker[['동별', '2021 년']].copy()
worker.columns = ['동별', '종사자수']
worker.sort_values('동별', inplace = True)
worker['동별'] = worker['동별'].replace('일원2동', '개포3동')
worker.sort_values('동별', inplace = True)
worker.reset_index(drop=True, inplace=True); worker

,동별,종사자수
0,가락1동,12367
1,가락2동,11544
2,가락본동,28151
3,가리봉동,4248
4,가산동,182376
...,...,...
421,효창동,2596
422,후암동,4605
423,휘경1동,3690
424,휘경2동,5457


In [6]:
work_com = work_company[['동별', '2021 년']].copy()
work_com.columns = ['동별', '사업체수']
work_com['동별'] = work_com['동별'].replace('일원2동', '개포3동')
work_com.sort_values('동별', inplace = True)
work_com.reset_index(drop=True, inplace=True); work_com

,동별,사업체수
0,가락1동,3791
1,가락2동,3155
2,가락본동,5069
3,가리봉동,1189
4,가산동,25043
...,...,...
421,효창동,702
422,후암동,1367
423,휘경1동,1285
424,휘경2동,1665


In [7]:
## 주거지역

house = pd.read_csv('./data/서울시 주택종류별 주택 (동별) 통계.csv', skiprows=3)
house.drop(house[house['동별(3)'] == '소계'].index, axis=0, inplace = True)
house = house[['동별(3)', '소계']]
house.columns = ['동별', '집 개수']

house['동별'] = house['동별'].replace('일원2동', '개포3동')
house.sort_values('동별', inplace = True); house.reset_index(drop=True, inplace=True); house
house



,동별,집 개수
0,가락1동,9716
1,가락2동,10424
2,가락본동,8410
3,가리봉동,2551
4,가산동,6200
...,...,...
420,효창동,3463
421,후암동,5363
422,휘경1동,4327
423,휘경2동,6802


In [8]:
# 소득


money = pd.read_csv('./data/서울시 상권분석서비스(소득소비-행정동).csv', encoding = 'cp949')
money = money[['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '월_평균_소득_금액', '지출_총금액']]
money = money[money['기준_년분기_코드'] == 20231]
money = money[['행정동_코드_명', '월_평균_소득_금액']].copy(); money
money.columns = ['동별', '소득']
money['동별'] = money['동별'].replace('일원2동', '개포3동')
money.sort_values('동별', inplace=True); money.reset_index(drop=True, inplace=True); money
money

,동별,소득
0,가락1동,4973588
1,가락2동,3910198
2,가락본동,3521561
3,가리봉동,2336331
4,가산동,2567594
...,...,...
420,효창동,3523201
421,후암동,3013677
422,휘경1동,2722774
423,휘경2동,3071586


In [9]:
# 교통혼잡도

traffic = pd.read_csv('./data/행정동별 교통혼잡시 평균속도 .csv')
traffic['emd_code'] = traffic['emd_code'].astype('str')
seoul_traffic = traffic[traffic['emd_code'].str[:1] == '1']
seoul_traffic

,sido_code,sigungu_code,emd_code,week_type,velocity_CG_AVRG
0,11000,11010,1101053,weekday,19.12
1,11000,11010,1101054,weekday,18.28
2,11000,11010,1101055,weekday,19.05
3,11000,11010,1101056,weekday,18.53
4,11000,11010,1101057,weekday,12.86
...,...,...,...,...,...
419,11000,11250,1125070,weekday,14.57
420,11000,11250,1125071,weekday,16.37
421,11000,11250,1125072,weekday,15.60
422,11000,11250,1125073,weekday,21.15


In [10]:
## 학생 비율, 노인 비율

age = pd.read_csv('./data/행정동별 연령 인구.csv', encoding = 'utf-8')
age = age[age['동별'].str[-1] == '동']

age2 = age[(age['연령별'] != '합계') & (age['항목'] == '계')]

age2.reset_index(drop=True, inplace = True)
age2.columns = ['동별', '연령별', '항목', '단위', '인구수']

age2



,동별,연령별,항목,단위,인구수
0,사직동,0~4세,계,NaN,178
1,사직동,5~9세,계,NaN,310
2,사직동,10~14세,계,NaN,313
3,사직동,15~19세,계,NaN,356
4,사직동,20~24세,계,NaN,420
...,...,...,...,...,...
8941,상일2동,80~84세,계,NaN,235
8942,상일2동,85~89세,계,NaN,133
8943,상일2동,90~94세,계,NaN,56
8944,상일2동,95~99세,계,NaN,16


In [11]:
consent = age2['동별'].unique()
df = pd.DataFrame(consent, columns = ['동별'])
age_groups = age2['연령별'].unique()
age_groups.sort()
age_groups

for age_group in age_groups:
    subset = age2[age2['연령별'] == age_group].drop_duplicates(subset='동별')
    df[age_group] = subset['인구수'].values

age_df = df
age_df.shape
age_df.iloc[:, 1:22].head()

,0~4세,100세 이상,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,...,55~59세,5~9세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세
0,178,5,313,356,420,666,811,685,733,750,...,696,310,727,572,369,329,302,200,70,19
1,36,2,105,95,102,153,161,157,228,176,...,184,72,216,190,140,119,91,58,23,4
2,117,4,393,458,581,681,667,592,710,737,...,791,268,729,613,391,354,275,146,57,19
3,411,12,737,795,969,1094,1053,1058,1324,1215,...,1517,622,1571,1252,725,664,469,348,99,36
4,207,4,471,459,437,392,389,466,626,677,...,592,355,619,480,342,255,259,116,41,7


In [12]:
## 연령별 나이

age_df['합계'] = age_df.iloc[:,1:22].sum(axis=1)
age_df['10대 비율'] = (age_df['10~14세'] + age_df['15~19세']) / df['합계']
age_df['20대 비율'] = (age_df['20~24세'] + age_df['25~29세']) / df['합계']
age_df['3040 비율'] = (age_df['30~34세'] + age_df['35~39세'] + age_df['40~44세'] + age_df['45~49세']) / age_df['합계']
age_df['65세 이상 비율'] = age_df.loc[:, '65~69세':'95~99세'].sum(axis=1) / age_df['합계']; age_df

age_data = age_df[['동별', '10대 비율', '20대 비율', '3040 비율', '65세 이상 비율']].copy(); age_data
age_data['동별'] = age_data['동별'].replace('일원2동', '개포3동')
age_data.sort_values('동별', inplace=True); age_data.reset_index(drop=True, inplace=True)

In [13]:
## 지하철역 개수

subway = pd.read_csv('./data/서울시 지하철역 정보.csv', encoding = 'cp949')
subway




,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
766,154,종로5가,1호선,37.570926,127.001849
767,153,종로3가,1호선,37.570406,126.991847
768,152,종각,1호선,37.570161,126.982923
769,151,시청,1호선,37.565715,126.977088


In [15]:
boundary = gpd.read_file('./data/행정동경계', encoding='cp949')
boundary['ADM_CD'] = boundary['ADM_CD'].astype('str')
seoul_boundary = boundary[boundary['ADM_CD'].str[0] == '1']
seoul_boundary

,BASE_DATE,ADM_CD,ADM_NM,geometry
0,20220630,11010530,사직동,"POLYGON ((197702.069 553187.311, 197703.431 55..."
1,20220630,11010540,삼청동,"POLYGON ((198170.457 553770.678, 198172.189 55..."
2,20220630,11010550,부암동,"POLYGON ((196621.023 556395.880, 196628.323 55..."
3,20220630,11010560,평창동,"POLYGON ((197800.719 559064.245, 197782.581 55..."
4,20220630,11010570,무악동,"POLYGON ((196444.745 553384.564, 196471.618 55..."
...,...,...,...,...
421,20220630,11250730,천호2동,"POLYGON ((210279.972 550065.332, 210287.877 55..."
422,20220630,11250740,길동,"POLYGON ((213117.764 549557.375, 213117.393 54..."
423,20220630,11250750,강일동,"POLYGON ((215607.323 553294.531, 215601.339 55..."
424,20220630,11250760,상일1동,"POLYGON ((215364.723 549613.445, 215368.026 54..."


In [16]:
from shapely.geometry import Point, Polygon, LineString

subway['geometry'] = subway.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
subway

,역사_ID,역사명,호선,위도,경도,geometry
0,9996,미사,5호선,37.560927,127.193877,POINT (127.193877 37.560927)
1,9995,강일,5호선,37.557490,127.175930,POINT (127.17593 37.55749)
2,4929,김포공항,김포골드라인,37.562360,126.801868,POINT (126.801868 37.56236)
3,4928,고촌,김포골드라인,37.601243,126.770345,POINT (126.770345 37.601243)
4,4927,풍무,김포골드라인,37.612488,126.732387,POINT (126.732387 37.612488)
...,...,...,...,...,...,...
766,154,종로5가,1호선,37.570926,127.001849,POINT (127.001849 37.570926)
767,153,종로3가,1호선,37.570406,126.991847,POINT (126.991847 37.570406)
768,152,종각,1호선,37.570161,126.982923,POINT (126.982923 37.570161)
769,151,시청,1호선,37.565715,126.977088,POINT (126.977088 37.565715)


In [17]:
subway = gpd.GeoDataFrame(subway)

In [18]:
subway.set_crs(epsg=4326, inplace=True)
subway.to_crs(epsg=5186, inplace = True)

In [21]:
subway_dong = gpd.sjoin(subway, seoul_boundary)
subway_dong['ADM_NM']

1       강일동
326     강일동
2      방화2동
102    방화2동
144    방화2동
       ... 
755    신당5동
759     소공동
760     소공동
769     소공동
761    숭인1동
Name: ADM_NM, Length: 402, dtype: object

In [17]:
subway_count = subway_dong.groupby('ADM_NM').size()
sub_count = pd.DataFrame(subway_count)

sub_count.reset_index(inplace = True)
sub_count.columns = ['행정동', '지하철역 개수']
sub_count['행정동'] = sub_count['행정동'].replace('일원2동', '개포3동')
sub_count.sort_values('행정동', inplace = True); sub_count.reset_index(drop = True, inplace = True)

In [18]:
## 행정동별 버스정류장 개수

bus_loc = pd.read_csv('./data/서울시버스정류소위치정보(20240110).csv')
bus_loc


bus_loc['geometry'] = bus_loc.apply(lambda row : Point(row['X좌표'], row['Y좌표']), axis=1)
bus_loc = gpd.GeoDataFrame(bus_loc); bus_loc

bus_loc.set_crs(epsg=4326, inplace = True)
bus_loc.to_crs(epsg=5186, inplace = True)

bus_count = gpd.sjoin(bus_loc, seoul_boundary)

bus_count

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입,geometry,index_right,BASE_DATE,ADM_CD,ADM_NM
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로,POINT (198918.001 552251.750),7,20220630,11010610,종로1·2·3·4가동
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,POINT (198905.685 552115.650),7,20220630,11010610,종로1·2·3·4가동
9,100000385,1010,광화문,126.977880,37.570240,중앙차로,POINT (198045.889 552300.092),7,20220630,11010610,종로1·2·3·4가동
10,100000386,1011,종로1가,126.981066,37.570148,중앙차로,POINT (198327.329 552289.885),7,20220630,11010610,종로1·2·3·4가동
11,100000387,1012,종로1가,126.981628,37.570271,중앙차로,POINT (198376.998 552303.500),7,20220630,11010610,종로1·2·3·4가동
...,...,...,...,...,...,...,...,...,...,...,...
11211,124000265,25650,강동농협,127.135338,37.548070,일반차로,POINT (211959.641 549847.952),413,20220630,11250610,천호1동
11212,124000434,25651,강동농협,127.135904,37.548400,일반차로,POINT (212009.658 549884.612),413,20220630,11250610,천호1동
11214,124000269,25654,천호초교입구사거리,127.134213,37.546491,일반차로,POINT (211860.476 549672.541),413,20220630,11250610,천호1동
11253,124000399,25782,굽은다리역,127.142744,37.545589,일반차로,POINT (212614.501 549573.500),413,20220630,11250610,천호1동


In [19]:
##

station = pd.DataFrame(bus_count.groupby(['ADM_CD', 'ADM_NM']).size())
station.reset_index(inplace = True)
station.columns = ['행정동코드', '행정동명', '버스정류장 개수']
station['행정동명'] = station['행정동명'].replace('일원2동', '개포3동')
station['행정동명'] = station['행정동명'].replace('상일1동', '상일동')
station['행정동명'] = station['행정동명'].replace('상일2동', '상일동')

station.sort_values('행정동명', inplace = True)
station.reset_index(inplace = True, drop =True)
station.iloc[246, 1] = '신사동1'
station.iloc[247, 1] = '신사동2'
station = station.groupby('행정동명').sum()
station.reset_index(inplace = True); station.head()
station

,행정동명,버스정류장 개수
0,가락1동,14
1,가락2동,21
2,가락본동,25
3,가리봉동,9
4,가산동,105
...,...,...
420,효창동,15
421,후암동,16
422,휘경1동,13
423,휘경2동,18


In [20]:
## 자동차 등록대수
car = pd.read_csv('./data/서울시 행정동별 자동차 등록대수 현황.csv', encoding = 'cp949')
car = car.groupby('읍면동 (행정동)').sum()
car.reset_index(inplace = True)
car['읍면동 (행정동)'] = car['읍면동 (행정동)'].str.strip()
car = car[car['읍면동 (행정동)'].str[-1] == '동']
car['읍면동 (행정동)'] = car['읍면동 (행정동)'].str.split(' ').str[1]
car.sort_values('읍면동 (행정동)', inplace = True)
car.reset_index(inplace = True, drop=True)
car[car['읍면동 (행정동)'] == '신사동']
car.iloc[262,0] = '신사동1'
car.iloc[263,0] = '신사동2'
car = car[['읍면동 (행정동)', '계']]
car.columns = ['읍면동', '자동차 등록대수']
car['읍면동'] = car['읍면동'].replace('일원2동', '개포3동')
car['읍면동'] = car['읍면동'].str.replace('.', '·' )
car


C:\Users\KJY\AppData\Local\Temp\ipykernel_17272\2106501563.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  car['읍면동'] = car['읍면동'].str.replace('.', '·' )


,읍면동,자동차 등록대수
0,가락1동,11406.0
1,가락2동,10758.0
2,가락본동,12586.0
3,가리봉동,2730.0
4,가산동,20010.0
...,...,...
444,효창동,3246.0
445,후암동,4014.0
446,휘경1동,3932.0
447,휘경2동,6353.0


In [21]:
# 서울트래픽, 지하철역 개수, 행정동별 버스 노선 개수, 상일1, 2동 병합
# , house, money, age_data, station]






df2 = pd.concat([pop, work_com, worker], axis=1)
df2
df2.loc[df2[df2['행정동'].str.contains('상일')].index, '행정동'] = '상일동'


df2[['사업체수','종사자수']] = df2[['사업체수', '종사자수']].astype('int64')

df2 = df2.groupby(['행정구','행정동']).sum()
df2.reset_index(inplace = True)
df2.sort_values('행정동', inplace = True)
df2.reset_index(inplace = True, drop=True)
df2

df3 = pd.concat([df2, house, money, age_data, station], axis=1)
df3.drop('동별', axis=1, inplace = True)

df4 = pd.merge(df3, sub_count, how='left')
df4


df5 = pd.merge(df4, car,left_on = '행정동명', right_on ='읍면동', how='left')

df5['지하철역 개수'] = df5['지하철역 개수'].fillna(0)



df5.shape


(425, 16)

In [22]:
# 교통혼잡도

traffic = pd.read_csv('./data/행정동별 교통혼잡시 평균속도 .csv')
traffic['emd_code'] = traffic['emd_code'].astype('str')
seoul_traffic = traffic[traffic['emd_code'].str[:1] == '1']
seoul_traffic.head()

,sido_code,sigungu_code,emd_code,week_type,velocity_CG_AVRG
0,11000,11010,1101053,weekday,19.12
1,11000,11010,1101054,weekday,18.28
2,11000,11010,1101055,weekday,19.05
3,11000,11010,1101056,weekday,18.53
4,11000,11010,1101057,weekday,12.86


In [23]:
dong = gpd.read_file('./data/HangJeongDong_ver20230701.geojson')

In [24]:
pd.set_option('display.max_rows',10)

df = seoul_traffic.merge(dong, left_on='emd_code', right_on='adm_cd', how='left')
df.loc[df[df['emd_code'] == '1123074'].index, 'temp'] = '강남구 개포3동'
df.loc[df[df['emd_code'] == '1125051'].index, 'temp'] = '강동구 강일동'
df.loc[df[df['emd_code'] == '1125052'].index, 'temp'] = '강동구 상일동'


congestion = df[['temp', 'velocity_CG_AVRG']]
congestion.columns = ['행정동', '도로 평균 속도']

congestion['행정구'] = congestion['행정동'].copy().str.split(' ').str[0]
congestion['행정동'] = congestion['행정동'].copy().str.split(' ').str[1]
congestion



C:\Users\KJY\AppData\Local\Temp\ipykernel_17272\634278005.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  congestion['행정구'] = congestion['행정동'].copy().str.split(' ').str[0]
C:\Users\KJY\AppData\Local\Temp\ipykernel_17272\634278005.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  congestion['행정동'] = congestion['행정동'].copy().str.split(' ').str[1]


,행정동,도로 평균 속도,행정구
0,사직동,19.12,종로구
1,삼청동,18.28,종로구
2,부암동,19.05,종로구
3,평창동,18.53,종로구
4,무악동,12.86,종로구
...,...,...,...
419,둔촌1동,14.57,강동구
420,둔촌2동,16.37,강동구
421,암사1동,15.60,강동구
422,천호2동,21.15,강동구


In [25]:
congestion.sort_values('행정동', inplace = True)
congestion.reset_index(inplace = True, drop=True)
congestion.head()

C:\Users\KJY\AppData\Local\Temp\ipykernel_17272\322016502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  congestion.sort_values('행정동', inplace = True)


,행정동,도로 평균 속도,행정구
0,가락1동,16.42,송파구
1,가락2동,15.61,송파구
2,가락본동,16.24,송파구
3,가리봉동,15.08,구로구
4,가산동,18.46,금천구


In [26]:
congestion['행정동'] = congestion['행정동'].str.replace('·', '.')
df10 = pd.merge(df5, congestion, left_on=['행정구', '행정동'], right_on=['행정구', '행정동'], how = 'left')
df10.fillna(df10['도로 평균 속도'].mean(), inplace = True)
df10.drop(['읍면동'], axis=1, inplace = True)
df10.to_csv('독립변수 정리.csv', encoding = 'cp949')
df10

C:\Users\KJY\AppData\Local\Temp\ipykernel_17272\1691265517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  congestion['행정동'] = congestion['행정동'].str.replace('·', '.')


,행정구,행정동,인구수,사업체수,종사자수,집 개수,소득,10대 비율,20대 비율,3040 비율,65세 이상 비율,행정동명,버스정류장 개수,지하철역 개수,자동차 등록대수,도로 평균 속도
0,송파구,가락1동,27670,3791,12367,9716,4973588,0.096314,0.095699,0.354355,0.142212,가락1동,14,3.0,11406.0,16.42
1,송파구,가락2동,31450,3155,11544,10424,3910198,0.103529,0.130175,0.310715,0.149571,가락2동,21,2.0,10758.0,15.61
2,송파구,가락본동,25617,5069,28151,8410,3521561,0.063552,0.140571,0.318070,0.164930,가락본동,25,1.0,12586.0,16.24
3,구로구,가리봉동,12478,1189,4248,2551,2336331,0.025325,0.146818,0.285062,0.178875,가리봉동,9,0.0,2730.0,15.08
4,금천구,가산동,27166,25043,182376,6200,2567594,0.025068,0.268571,0.376058,0.110248,가산동,105,3.0,20010.0,18.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,용산구,효창동,10828,702,2596,3463,3523201,0.062615,0.152290,0.337828,0.147950,효창동,15,0.0,3246.0,16.11
421,용산구,후암동,16540,1367,4605,5363,3013677,0.068622,0.137485,0.320798,0.185248,후암동,16,0.0,4014.0,18.46
422,동대문구,휘경1동,16168,1285,3690,4327,2722774,0.055418,0.348714,0.244805,0.135267,휘경1동,13,1.0,3932.0,22.15
423,동대문구,휘경2동,25288,1665,5457,6802,3071586,0.077349,0.261152,0.251028,0.167550,휘경2동,18,0.0,6353.0,15.97


In [32]:
bus_use = pd.read_csv('./data/서울시 행정동별 버스 총 승차 승객수 정보.csv', encoding = 'cp949')

In [189]:
bus_use['기준_날짜'] = bus_use['기준_날짜'].astype('str')
bus_2023 = bus_use[bus_use['기준_날짜'].str.startswith('2023')]
bus_dong = bus_2023.groupby('행정동_ID').sum()
bus_dong = bus_dong.reset_index()
bus_dong = bus_dong[['행정동_ID', '버스_승객_수']]
bus_dong = bus_dong[bus_dong['버스_승객_수'] != 0]

bus_dong['행정동_ID'] = bus_dong['행정동_ID'].astype('str')


,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시
0,20240115,1118060,7222.0,3.0,0.0,0.0,0.0,208.0,390.0,463.0,...,321.0,270.0,341.0,301.0,300.0,246.0,128.0,102.0,71.0,27.0
1,20240115,1118058,5733.0,6.0,0.0,0.0,0.0,18.0,156.0,363.0,...,262.0,273.0,266.0,265.0,322.0,176.0,125.0,78.0,98.0,54.0
2,20240115,1118055,11131.0,34.0,10.0,7.0,11.0,225.0,358.0,477.0,...,474.0,560.0,577.0,776.0,1090.0,675.0,434.0,342.0,283.0,104.0
3,20240115,1118053,9622.0,21.0,0.0,3.0,4.0,176.0,334.0,500.0,...,416.0,409.0,439.0,518.0,620.0,430.0,275.0,232.0,195.0,108.0
4,20240115,1118051,35904.0,71.0,17.0,9.0,17.0,125.0,463.0,939.0,...,1267.0,1440.0,1715.0,3168.0,4850.0,3045.0,1965.0,1556.0,933.0,446.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314785,20220101,1110061,2480.0,5.0,0.0,0.0,0.0,5.0,43.0,66.0,...,193.0,212.0,196.0,195.0,119.0,112.0,69.0,112.0,38.0,30.0
314786,20220101,1110060,3485.0,19.0,0.0,0.0,1.0,24.0,78.0,79.0,...,291.0,313.0,278.0,243.0,196.0,126.0,120.0,148.0,86.0,43.0
314787,20220101,1110056,2377.0,1.0,0.0,0.0,0.0,7.0,54.0,83.0,...,218.0,215.0,192.0,176.0,125.0,76.0,72.0,61.0,34.0,14.0
314788,20220101,1115069,2229.0,6.0,0.0,0.0,0.0,4.0,23.0,61.0,...,189.0,144.0,174.0,201.0,165.0,83.0,80.0,159.0,128.0,39.0


In [201]:
bus_2023[bus_2023['행정동_ID'] == '1117068']

,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시


In [72]:
for index, row in bus_dong.iterrows():
    if len(row['행정동_ID']) == 7:
        bus_dong.at[index, '행정동_ID'] = row['행정동_ID'] + '0'
    else:
        bus_dong.at[index, '행정동_ID'] = row['행정동_ID']

In [93]:
pd.set_option('display.max_rows', 10)

bus_dong = bus_dong.reset_index(drop =True); bus_dong

,행정동_ID,버스_승객_수
0,11010530,9308863.0
1,11010540,1556254.0
2,11010550,3438608.0
3,11010560,3042777.0
4,11010570,678600.0
...,...,...
422,11250730,3659833.0
423,11250740,3327375.0
424,11170730,675844.0
425,11250550,749908.0


In [127]:
dong_code = pd.read_excel('./data/adm_code (1).xls', skiprows=1)
dong_code

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,10,종로구,530,사직동
1,11,서울특별시,10,종로구,540,삼청동
2,11,서울특별시,10,종로구,550,부암동
3,11,서울특별시,10,종로구,560,평창동
4,11,서울특별시,10,종로구,570,무악동
...,...,...,...,...,...,...
3523,39,제주특별자치도,20,서귀포시,580,서홍동
3524,39,제주특별자치도,20,서귀포시,590,대륜동
3525,39,제주특별자치도,20,서귀포시,600,대천동
3526,39,제주특별자치도,20,서귀포시,610,중문동


In [188]:
bus_dong[bus_dong['행정동_ID'] == '11170680']

,행정동_ID,버스_승객_수
270,11170680,3386518.0


In [183]:
dong_code['행정동코드'] = dong_code['시도코드'] *1000000 + dong_code['시군구코드'] * 1000  + dong_code['읍면동코드']
dong_code['행정동코드'] = dong_code['행정동코드'].astype('str')
dong_code


dong_code[dong_code['행정동코드'] == '11170740']

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭,행정동코드
275,11,서울특별시,170,구로구,740,항동,11170740


In [186]:
df11 = pd.merge(bus_dong, dong_code, left_on='행정동_ID', right_on = '행정동코드', how='left')

df11[df11.isnull().any(axis=1)]



,행정동_ID,버스_승객_수,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭,행정동코드
270,11170680,3386518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,11230740,879456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,11250510,3087037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,11250520,2926401.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df11.loc[270, '읍면동명칭'] = '오류2동' #11170680
df11.loc[372, '읍면동명칭'] = '개포3동'
df11.loc[406, '읍면동명칭'] = '강일1동'
df11.loc[407, '읍면동명칭'] = '강일2동'